### Entrenaremos un modelo en Vertex AI haciendo uso de su infraestructura.

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

In [23]:
!python --version

Python 3.9.2


### Primero definiremos un modelo encargado de entrenar datos ficticios
#### El propósito es la implementación del entrenamiento en la nube
Para definir el modelo, podemos usar alguna de las tres formas de construir modelos en TensorFlow:
* Modelo secuencial
* Modelo funcional
* Modelo por subclassing

In [13]:
# Modelo Funcional
x0 = tf.keras.Input(shape=(10,))  
x = tf.keras.layers.Dense(10)(x0)
x = tf.keras.layers.Dense(1)(x)
model_functional = tf.keras.Model(x0, x)  

# Modelo Secuencial
model_sequential = tf.keras.Sequential([
    tf.keras.layers.Dense(10),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Dense(1),
])

# Subclassing
class MyModel(tf.keras.Model):  # Hereda de tf.keras.Model.
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense = tf.keras.layers.Dense(10)

    def call(self, x):
        return self.dense(x)

class MyModel2(MyModel):  # Hereda de MyModel.
    def __init__(self):
        super(MyModel2, self).__init__()
        self.dense2 = tf.keras.layers.Dense(10)
        self.dense3 = tf.keras.layers.Dense(1)

    def call(self, x):
        x = super().call(x)  # Llama al método call de MyModel.
        x = self.dense2(x)
        return self.dense3(x)

In [14]:
def custom_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)  # Convierte y_true a float32
    error = y_pred - y_true
    return tf.reduce_mean(tf.square(error))

strategy = tf.distribute.get_strategy()  # Use the Default Strategy
#strategy = tf.distribute.MirroredStrategy() # varias GPUs en una máquina.
# strategy = tf.distribute.MultiWorkerMirroredStrategy() # múltiples máquinas, cada una con una o varias GPUs.
# Abrir un scope de estrategia
with strategy.scope():
    # Todo lo que crees dentro de este bloque será distribuido
    model = MyModel2()
    model.compile(optimizer='adam',
                  loss=custom_loss,
                  metrics=['accuracy'])

- También crearemos un ModelCheckpoint para guardar el modelo, ya sea en local o en la nube. Le diremos que solo guarde el mejor, basado en una métrica.

- Probaremos el código para asegurarnos de que funcionará en la nube.

In [15]:
# Crear un callback de ModelCheckpoint
checkpoint = ModelCheckpoint('model/m0',
                             monitor='accuracy',  # la métrica a monitorizar
                             verbose=1,  # log level
                             save_best_only=True,  # solo guarda el mejor modelo
                             mode='max',  # modo 'max' porque queremos maximizar la accuracy
                             save_format='tf')  # especifica que se debe usar el formato SavedModel

x_train = np.random.random((1000, 10))
y_train = np.random.randint(2, size=(1000,))

model.fit(x_train, y_train,
          epochs=1,
          batch_size=32,
          callbacks=[checkpoint])  # asegúrate de pasar el callback aquí

# Evaluar el modelo
x_test = np.random.random((200, 10))
y_test = np.random.randint(2, size=(200,))
evv = model.evaluate(x_test, y_test)
print(evv)
model.save('model/m1')

 1/32 [..............................] - ETA: 11s - loss: 2.4936 - accuracy: 0.4688
Epoch 1: accuracy improved from -inf to 0.46900, saving model to model/m0
INFO:tensorflow:Assets written to: model/m0/assets


INFO:tensorflow:Assets written to: model/m0/assets


7/7 [==============================] - 0s 2ms/step - loss: 0.8309 - accuracy: 0.5150
[0.8308799266815186, 0.5149999856948853]
INFO:tensorflow:Assets written to: model/m1/assets


INFO:tensorflow:Assets written to: model/m1/assets


- Creamos una carpeta que contendrá los archivos para nuestra imagen Docker. 

- Estos constan de un Dockerfile, requirements.txt y el script train.py

In [19]:
!mkdir -p train_dir

In [35]:
%%writefile train_dir/train.py
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

# Modelo Secuencial
model_sequential = tf.keras.Sequential([
    tf.keras.layers.Dense(10),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Dense(1),
])

strategy = tf.distribute.get_strategy() 
#strategy = tf.distribute.MirroredStrategy()  # varias GPUs en una máquina.
# Abrir un scope de estrategia
with strategy.scope():
    model = model_sequential

    # Compilar el modelo
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

# Generar algunos datos de ejemplo
x_train = np.random.random((1000, 10))
y_train = np.random.randint(2, size=(1000,))

# Crear un callback de ModelCheckpoint
checkpoint = ModelCheckpoint('gs://models_ai_save/best_model',
                             monitor='accuracy',  # la métrica a monitorizar
                             verbose=1,  # log level
                             save_best_only=True,  # solo guarda el mejor modelo
                             mode='max',  # modo 'max' porque queremos maximizar la accuracy
                             save_format='tf')  # especifica que se debe usar el formato SavedModel

# Entrenar el modelo
model.fit(x_train, y_train, epochs=3, batch_size=32, callbacks=[checkpoint])

# Evaluar el modelo
x_test = np.random.random((200, 10))
y_test = np.random.randint(2, size=(200,))
evv = model.evaluate(x_test, y_test)
print(evv)

# Guardar el modelo final en el bucket de GCS
model.save('gs://models_ai_save/model_final')

Overwriting train_dir/train.py


In [36]:
%%writefile train_dir/Dockerfile

# Usa la imagen base de TensorFlow con soporte para GPU
FROM python:3.9.2

# Actualiza pip
RUN pip install --upgrade pip

# Copia el archivo requirements.txt al contenedor
COPY requirements.txt /requirements.txt

# Instala las dependencias
RUN pip install -r /requirements.txt

# Copia el script de entrenamiento al contenedor
COPY train.py /train.py

# Establece el script de entrenamiento como el comando predeterminado del contenedor
CMD ["python", "/train.py"]


Overwriting train_dir/Dockerfile


In [37]:
%%writefile train_dir/.dockerignore

__pycache__
*.pyc
*.pyo
*.egg-info
**/model/m0/
**/model/m1/

env/
.git
.dockerignore
.gitignore

# Jupyter
.ipynb_checkpoints/

# Python
__pycache__/
*.pyc
*.pyo
*.egg-info
*.egg

# Entornos virtuales
venv/
.env/
env
**/__pycache__
__pycache__
*.virtualenv/

# Generales
.DS_Store
*.log
.idea/

Overwriting train_dir/.dockerignore


- Una vez con los archivos en la carpeta, desde la terminal deberemos construir la imagen Docker, etiquetarla y subirla a nuestro repositorio en Artifact Registry. 

- Esto se debe hacer en una terminal fuera de Jupyter Notebook, ya que Jupyter no permite la ejecución de Docker sobre él.

# Ejecutamos en terminal

### docker build -t train_model .
### docker tag train_model us-central1-docker.pkg.dev/projecto2-373519/myimages/train_model:v1.0.0
### docker push us-central1-docker.pkg.dev/projecto2-373519/myimages/train_model:v1.0.0

* Una vez con la imagen en la nube, ejecutamos el siguiente comando para crear un custom job o entrenamiento personalizado. 
* En este caso, le damos una ubicación, un nombre, el tipo de máquina y las réplicas, mayor a 1 en caso de usar estrategias distribuidas.

In [39]:
!gcloud ai custom-jobs create \
  --region=us-central1 \
  --display-name=entrenamiento_prueba_2 \
  --worker-pool-spec=machine-type=n1-standard-4,replica-count=1,container-image-uri=us-central1-docker.pkg.dev/projecto2-373519/myimages/train_model_cpu:v1.0.0


Using endpoint [https://us-central1-aiplatform.googleapis.com/]
CustomJob [projects/936401695274/locations/us-central1/customJobs/3065650967581032448] is submitted successfully.

Your job is still active. You may view the status of your job with the command

  $ gcloud ai custom-jobs describe projects/936401695274/locations/us-central1/customJobs/3065650967581032448

or continue streaming the logs with the command

  $ gcloud ai custom-jobs stream-logs projects/936401695274/locations/us-central1/customJobs/3065650967581032448


In [40]:
!gcloud ai custom-jobs describe projects/936401695274/locations/us-central1/customJobs/3065650967581032448

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
createTime: '2023-09-28T08:40:55.110080Z'
displayName: entrenamiento_prueba_2
jobSpec:
  workerPoolSpecs:
  - containerSpec:
      imageUri: us-central1-docker.pkg.dev/projecto2-373519/myimages/train_model_cpu:v1.0.0
    diskSpec:
      bootDiskSizeGb: 100
      bootDiskType: pd-ssd
    machineSpec:
      machineType: n1-standard-4
    replicaCount: '1'
name: projects/936401695274/locations/us-central1/customJobs/3065650967581032448
startTime: '2023-09-28T08:40:55.261758Z'
state: JOB_STATE_PENDING
updateTime: '2023-09-28T08:41:15.974667Z'


# Monitoreo del entrenamiento

In [41]:
!gcloud ai custom-jobs stream-logs projects/936401695274/locations/us-central1/customJobs/3065650967581032448

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
INFO	2023-09-28 02:40:55 -0600	service	Waiting for job to be provisioned.
ERROR	2023-09-28 02:42:08 -0600	workerpool0-0	2023-09-28 08:42:08.856615: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
ERROR	2023-09-28 02:42:08 -0600	workerpool0-0	2023-09-28 08:42:08.915167: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
ERROR	2023-09-28 02:42:08 -0600	workerpool0-0	2023-09-28 08:42:08.915250: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
ERROR	2023-09-28 02:42:08 -0600	workerpool0-0	2023-09-28 08:42:08.915299: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS

# FIN